In [42]:
import torch

blank_ = 128
alphabet_size = 129


for i in range(645):
    if (i - blank_) % alphabet_size == 0:
        d = int((i - blank_) / alphabet_size)
        print(i, ',', d)

128 , 0
257 , 1
386 , 2
515 , 3
644 , 4


In [58]:
import torch

blank_ = 128
alphabet_size = 129


for i in range(645):
    if (i - 0) % alphabet_size == 0:
        d = int((i - 0) / alphabet_size)
        print(i, ',', d)

0 , 0
129 , 1
258 , 2
387 , 3
516 , 4


In [ ]:
T = 100
for t in range(T, 0, -1):
    for i in range(645):
        if (i - blank_) % alphabet_size == 0:
            d = int((i - blank_) / alphabet_size)
            if t < T - d and d > 0:
                print(i, ",", d, '|', t)

In [ ]:
T = 100
for t in range(T, 0, -1):
    for i in range(645):
        if (i - 30) % alphabet_size == 0:
            d = int(((i - 30) / alphabet_size))
            if t + d < T:
                print(i, ',', d, '|', t)

In [7]:
import torch

k = 129 + 128
k = 129 * 2 + 30


k % 129
k // 129

2

In [2]:
import torch

logits = torch.rand(1,1,1,645)

vocab_size = int(logits.size(-1) / 5)

print(0 % vocab_size)
print(0 // vocab_size)

0
0


In [ ]:
import torch

a = torch.rand(2,3,10)

# slice along the last dimension
# 0:5 and then 6:8
print(a)
b = a[:,:,0:5]
c = a[:,:,6:8]
d = torch.cat((b,c),dim=2)
print(d)
print(d.shape)


In [ ]:
import torch

a = torch.rand(2,3,4)
b = a.view([-1])
print(a)
print(b)
print(b.shape)

c = b.reshape(2,3,4)
print(c)

In [32]:
durations = [0,1,2,3,4]

alphabet_size = 129

(durations[1] + 1) * alphabet_size - 1
129 + 128


257

In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = '6,7'


import torch

a = torch.load('/home/jykang/NeMo/nemo_experiments/Conformer-TDT-BPE-Stateless-DEP/test3/2023-08-10_14-04-33--val_wer=0.0383-epoch=150-last_expanded.ckpt')
print(a.keys())
print(len(a['state_dict'].keys()))
print(a['state_dict'].keys())
print(a['optimizer_states'][0].keys())

print(a['optimizer_states'][0]['state'][605]['exp_avg_sq'].shape)
print(a['optimizer_states'][0]['state'][605]['exp_avg'].shape)

print(a['optimizer_states'][0]['state'][604]['exp_avg_sq'].shape)
print(a['optimizer_states'][0]['state'][604]['exp_avg'].shape)


dict_keys(['epoch', 'global_step', 'pytorch-lightning_version', 'state_dict', 'loops', 'callbacks', 'optimizer_states', 'lr_schedulers', 'hparams_name', 'hyper_parameters', 'hparams_type'])
656
dict_keys(['state', 'param_groups'])
torch.Size([134])
torch.Size([134])
torch.Size([134, 320])
torch.Size([134, 320])


In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = '6,7'


import torch

checkpoint = torch.load('/home/jykang/NeMo/nemo_experiments/Conformer-TDT-BPE-Stateless-DEP/test3/2023-08-10_14-04-33--val_wer=0.0383-epoch=150-last.ckpt')
state_dict = checkpoint['state_dict']
num_durations = 5

checkpoint['optimizer_states'][0]['state'][605]['exp_avg_sq'] = torch.rand(645)
checkpoint['optimizer_states'][0]['state'][605]['exp_avg'] = torch.rand(645)
checkpoint['optimizer_states'][0]['state'][604]['exp_avg_sq'] = torch.rand(645, 320)
checkpoint['optimizer_states'][0]['state'][604]['exp_avg'] = torch.rand(645, 320)


label_state = state_dict['joint.joint_net.2.weight'][:-num_durations, :] # (129, 320)
label_bais_state = state_dict['joint.joint_net.2.bias'][:-num_durations] # (129)

duration_state = state_dict['joint.joint_net.2.weight'][-num_durations:, :] # (5, 320)
duration_bais_state = state_dict['joint.joint_net.2.bias'][-num_durations:] # (5)

duration_state = duration_state.repeat_interleave(label_state.shape[0], dim=0) # (645, 320)
duration_bais_state = duration_bais_state.repeat_interleave(label_state.shape[0]) # (645)

label_state = label_state.repeat(num_durations, 1) # (645, 320)
label_bais_state = label_bais_state.repeat(num_durations) # (645)


new_state = label_state + duration_state # (645, 320)
new_bais_state = label_bais_state + duration_bais_state # (645)


new_state_dict = {}
for key in state_dict.keys():
    if key == 'joint.joint_net.2.weight':
        new_state_dict[key] = new_state
    elif key == 'joint.joint_net.2.bias':
        new_state_dict[key] = new_bais_state
    else:
        new_state_dict[key] = state_dict[key]


checkpoint['state_dict'] = new_state_dict
torch.save(checkpoint, '/home/jykang/NeMo/nemo_experiments/Conformer-TDT-BPE-Stateless-DEP/test3/2023-08-10_14-04-33--val_wer=0.0383-epoch=150-last_test3.ckpt')


In [ ]:
print(label_bais_state.shape)
print(label_bais_state[0] == label_bais_state[129])
print(label_bais_state[0] == label_bais_state[1])
print(label_state.shape)
print(label_state[0] == label_state[129])
print(label_state[0] == label_state[1])

In [ ]:
print(duration_bais_state.shape)
print(duration_bais_state[0] == duration_bais_state[129])
print(duration_bais_state[0] == duration_bais_state[1])
print(duration_state.shape)
print(duration_state[0] == duration_state[129])
print(duration_state[0] == duration_state[1])

In [39]:
expanded = torch.load('/home/jykang/NeMo/nemo_experiments/Conformer-TDT-BPE-Stateless-DEP/test3/2023-08-10_14-04-33--val_wer=0.0383-epoch=150-last_expanded.ckpt')


print(expanded['state_dict']['joint.joint_net.2.weight'].shape)
print(expanded['state_dict']['joint.joint_net.2.bias'].shape)   

torch.Size([645, 320])
torch.Size([645])


In [41]:
int((129-30)/129)

0